# Fake News Detection Arabic


> ## Install Requirements

In [10]:
!pip install farasapy
!pip install tqdm
!pip install gensim
!pip install spacy
!pip install scipy
!pip install wandb
!pip install nltk
!pip install xgboost 

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.6/60.6 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.6/38.6 MB 45.1 MB/s eta 0:00:00:00:0100:01
  Attempting uninstall: scipy
    Found existing installation: scipy 1.15.2
    Uninstalling scipy-1.15.2:
      Successfully uninstalled scipy-1.15.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tsfresh 0.21.0 requires scipy>=1.14.0; python_version >= "3.10", but you have scipy 1.13.1 which is incompatible.
nilearn 0.11.1 requires scikit-learn>=1.4.0, but you have scikit-learn 1.2.2 which is incompatible.
bigframes 1.36.0 requires rich<14,>=12.4.4, but you have rich 14.0.0 which is incompatible.
imbalanced-learn 0.13.0 requires scikit-learn<2,>=1.3.2, but you have scikit-learn 1.2.2 which is incompatible.
plotnine 0.14.5 requires matplotlib>=3.8.0, but you have matplotlib 3.7.5 which is inc

> ## Import Requirements

In [11]:
import pandas as pd 
import re
import string
import numpy as np
import matplotlib.pyplot as plt
import pickle
from time import time
from pathlib import Path
from sklearn.metrics import roc_auc_score, roc_curve
import matplotlib.pyplot as plt




import spacy
import nltk
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer
from nltk.tokenize import sent_tokenize
from nltk.tokenize import word_tokenize
from farasa.stemmer import FarasaStemmer
from farasa.segmenter import FarasaSegmenter
import unicodedata
from nltk.stem import WordNetLemmatizer
import joblib
from multiprocessing import Pool
from concurrent.futures import ThreadPoolExecutor

from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')


from sklearn.model_selection import train_test_split


from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier
from sklearn.svm import SVC
from xgboost import XGBClassifier

from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report , confusion_matrix, ConfusionMatrixDisplay

from gensim.models.fasttext import load_facebook_vectors



import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dropout, BatchNormalization, Dense, Bidirectional


import torch
from transformers import XLMRobertaTokenizer, XLMRobertaForSequenceClassification, Trainer, TrainingArguments
from datasets import Dataset

2025-04-28 20:53:58.032295: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1745873638.237336      31 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1745873638.296263      31 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


> ## Import Data

In [ ]:
df_ar =  pd.read_csv('/Cleaned_Data/processed_ar_data.csv')

In [17]:
df_ar

,title,text,label
0,"فيديو, هل لديك حساسية طعام؟المدة, 25,18",يعاني الكثير الشباب منطقة الشرق الأوسط وشمال أ...,real
1,اخر الاخبار اليوم محافظ المنيا ورئيس الجامعة ي...,الدكتور مصطفي عبد النبي رئيس جامعة المنيا والل...,fake
2,مدبولي يتابع الموقف التنفيذي لمشروع تطوير وتنم...,وأكد رئيس الوزراء المشروع القومي الكبير سيتم إ...,real
3,تسرب بسببها فصل بالكامل.. فاطمة رشدى ضربت الطا...,شكرا لقرائتكم خبر تسرب بسببها فصل بالكامل فاطم...,fake
4,سقوط تشكيل عصابي للاتجار بالمخدرات وحيازة الأس...,سقوط تشكيل عصابي للاتجار بالمخدرات وحيازة الأس...,real
...,...,...,...
365995,صحافة نت الإمارات : إنترفيو.. لوكاس كو: كوريا ...,صحافة نت الإمارات تفاصيل المقالة التالية إنترف...,fake
365996,اصابة الفنانة المصرية هنا الزاهد بكورونا,نقل السابع المصري نصدر مقرب النجمة الزاهد أنها...,real
365997,افتتاح معرض أهلا رمضان في مدينة ناصر بسوهاج بت...,كتب فهد فكرى بلوم افتتح اللواء طارق الفقي محاف...,real
365998,الوحدة الاخباري : وسائل إعلام: توقعات بفرض واش...,مصدران مطلعان الاثنين المتوقع تفرض واشنطن عقوب...,fake


In [ ]:
X = df_ar['text']
y = df_ar['label']

> ##  Vectorization (TFIDF)

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer()

X_tfidf = tfidf.fit_transform(X)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_tfidf, y, test_size=0.20 , random_state=42, stratify=y)

> ## Train ML Models

In [ ]:

models = {
    "Logistic Regression": LogisticRegression(),
    "Gradient Boosting": GradientBoostingClassifier(random_state=0),
    "Random Forest": RandomForestClassifier(random_state=0),
    "XGBoost": XGBClassifier(use_label_encoder=False, eval_metric='logloss', random_state=42),
    "SVM": SVC(probability=True)  
}

def train_and_evaluate_model(name, model, X_train, y_train, X_test, y_test):
    """Train a single model and evaluate its performance"""
    print(f"⏳ Training {name}...")
    start_time = time()
    
    # Train model
    model.fit(X_train, y_train)
    training_time = time() - start_time
    
    # Evaluate
    y_pred = model.predict(X_test)
    
    metrics = {
        'accuracy': accuracy_score(y_test, y_pred),
        'precision': precision_score(y_test, y_pred),
        'recall': recall_score(y_test, y_pred),
        'f1': f1_score(y_test, y_pred),
        'training_time': training_time
    }
    report = classification_report(y_test, y_pred)
    
    print(f"✅ {name} trained in {training_time:.2f}s")
    print(f"\n🔍 {name} Metrics:")
    print(f"  Training Time: {metrics['training_time']:.2f}s")
    print(f"  Accuracy: {metrics['accuracy']:.4f}")
    print(f"  Precision: {metrics['precision']:.4f}")
    print(f"  Recall: {metrics['recall']:.4f}")
    print(f"  F1 Score: {metrics['f1']:.4f}")
    print("Classification Report:")
    print(report)

    joblib.dump(model, f"./Models/Arabic/{name}_model.pkl")
    print(f"✅ {name} model saved as {name}_model.pkl")
    print("─" * 50)
    return model, name, metrics, report



# Train models sequentially
results = []
for name, model in models.items():
    result = train_and_evaluate_model(name, model, X_train, y_train, X_test, y_test)
    results.append(result)

⏳ Training Logistic Regression...
✅ Logistic Regression trained in 50.19s

🔍 Logistic Regression Metrics:
  Training Time: 50.19s
  Accuracy: 0.8335
  Precision: 0.8285
  Recall: 0.8426
  F1 Score: 0.8355
Classification Report:
              precision    recall  f1-score   support

           0       0.84      0.82      0.83     50774
           1       0.83      0.84      0.84     51111

    accuracy                           0.83    101885
   macro avg       0.83      0.83      0.83    101885
weighted avg       0.83      0.83      0.83    101885

✅ Logistic Regression model saved as Logistic Regression_model.pkl
──────────────────────────────────────────────────


> ## Word embedding ( Word2Vec)

Download the model from : 
https://www.kaggle.com/datasets/murtadhayaseen/ccar300bin

In [ ]:
word2vec = load_facebook_vectors('path/cc.ar.300.bin')

In [31]:
print(f"Vocabulary Size: {len(word2vec.index_to_key)}")
print(f"Vector Size: {word2vec.vector_size}")

Vocabulary Size: 2000000
Vector Size: 300


In [32]:
word = "كتاب"  
vector = word2vec[word]  

print(f"Vector for '{word}':\n{vector[:10]}...")

Vector for 'كتاب':
[ 0.03640094 -0.05246057 -0.00099724  0.08418036  0.16739112  0.01615874
  0.02734274 -0.07865718  0.16556144 -0.15287851]...


In [33]:
similar_words = word2vec.most_similar("كتاب", topn=5)  
print("Most similar words to 'كتاب':")
for word, similarity in similar_words:
    print(f"{word}: {similarity:.4f}")

Most similar words to 'كتاب':
وكتاب: 0.7156
لكتاب: 0.6849
الكتاب: 0.6621
بكتاب: 0.6513
كتابه: 0.6431


In [34]:
word1 = "امرأة"  
word2 = "رجل"  
similarity = word2vec.similarity(word1, word2)

print(f"Similarity between '{word1}' and '{word2}': {similarity:.4f}")

Similarity between 'امرأة' and 'رجل': 0.6814


Find a word that fits the analogy:
* "ملك" (King) - "رجل" (Man) + "امرأة" (Woman) ≈ ???
* (i.e., What is to "Woman" as "King" is to "Man"?)

In [35]:
analogy_result = word2vec.most_similar(positive=["ملك", "امرأة"], negative=["رجل"], topn=1)
print(f"Analogy result (King - Man + Woman): {analogy_result[0]}")

Analogy result (King - Man + Woman): ('ملكة', 0.5876614451408386)


In [36]:
analogy_result = word2vec.most_similar(positive=["طبيب", "امرأة"], negative=["رجل"], topn=1)
print(f"Analogy result : {analogy_result[0]}")

Analogy result : ('طبيبة', 0.7060946822166443)


In [37]:
analogy_result = word2vec.most_similar(positive=["أب", "امرأة"], negative=["رجل"], topn=1)
print(f"Analogy result : {analogy_result[0]}")

Analogy result : ('اب', 0.6286279559135437)


In [38]:
word = "ذكاء_اصطناعي"  
if word in word2vec:
    print(f"'{word}' exists in vocabulary.")
else:
    print(f"'{word}' NOT found in vocabulary.")

'ذكاء_اصطناعي' exists in vocabulary.


In [39]:
vector = word2vec["مستقبل"]  # Example word: "Future"
nearest_words = word2vec.similar_by_vector(vector, topn=3)
print(f"Closest words to 'مستقبل': {nearest_words}")

Closest words to 'مستقبل': [('مستقبل', 1.0), ('ومستقبل', 0.8226288557052612), ('لمستقبل', 0.7837035059928894)]


In [40]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences


In [ ]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(df_ar["text"])
sequences = tokenizer.texts_to_sequences(df_ar["text"])
word_index = tokenizer.word_index

X = pad_sequences(sequences, maxlen=300)



In [42]:
df_ar = df_ar.reset_index(drop=True)


In [ ]:
vocab_size = len(word_index) + 1
vocab_size

727974

In [ ]:
embedding_dim = word2vec.vector_size

embedding_matrix = np.zeros((vocab_size, embedding_dim))
for word, i in word_index.items():
    if word in word2vec:
        embedding_matrix[i] = word2vec[word]


y = df_ar['label'].values


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

> ## Train LSTM Model

In [46]:
model = Sequential([
    Embedding(
        input_dim=vocab_size,
        output_dim=embedding_dim,
        weights=[embedding_matrix],
        input_shape=(300,), 
        trainable=False  
    ),
    Bidirectional(LSTM(64, return_sequences=True)), 
    Dropout(0.5),  
    BatchNormalization(),
    LSTM(64),  
    Dropout(0.5),
    Dense(32, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')
])

I0000 00:00:1745872977.490319      31 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 15513 MB memory:  -> device: 0, name: Tesla P100-PCIE-16GB, pci bus id: 0000:00:04.0, compute capability: 6.0


In [ ]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    loss='binary_crossentropy',
    metrics=['accuracy' 
            , tf.keras.metrics.Precision(name='precision'),
             tf.keras.metrics.Recall(name='recall')
            ]
)

In [48]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding (Embedding)                │ (None, 300, 300)            │     218,392,200 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ bidirectional (Bidirectional)        │ (None, 300, 128)            │         186,880 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 300, 128)            │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization                  │ (None, 300, 128)            │             512 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_1 (LSTM)                        │ (None, 64)                  │          49,408 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_1 (Dropout)                  │ (None, 64)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 32)                  │           2,080 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_2 (Dropout)                  │ (None, 32)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 1)                   │              33 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 218,631,113 (834.01 MB)

 Trainable params: 238,657 (932.25 KB)

 Non-trainable params: 218,392,456 (833.10 MB)

In [ ]:
from sklearn.utils import class_weight
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

class_weights = class_weight.compute_class_weight(
    class_weight='balanced',  
    classes=np.unique(y_train),  
    y=y_train 
)

class_weight_dict = dict(enumerate(class_weights))

early_stopping = EarlyStopping(monitor='val_loss',  
                               patience=3, 
                               restore_best_weights=True)  

reduce_lr = ReduceLROnPlateau(monitor='val_loss', 
                               factor=0.2,  
                               patience=2,   
                               min_lr=1e-4,    
                               verbose=1)



X_train_final, X_val, y_train_final, y_val = train_test_split(X_train, y_train, test_size=0.1, random_state=42)

history = model.fit(
    X_train_final, y_train_final,
    validation_data=(X_val, y_val),
    epochs=50,
    batch_size=128,
    verbose=1,
    callbacks=[reduce_lr],
    class_weight=class_weight_dict
)



Epoch 1/50


I0000 00:00:1745872988.331601     281 cuda_dnn.cc:529] Loaded cuDNN version 90300


2293/2293 ━━━━━━━━━━━━━━━━━━━━ 160s 67ms/step - accuracy: 0.6018 - loss: 0.6264 - val_accuracy: 0.7455 - val_loss: 0.4530 - learning_rate: 0.0010
Epoch 2/50
2293/2293 ━━━━━━━━━━━━━━━━━━━━ 152s 66ms/step - accuracy: 0.7456 - loss: 0.4486 - val_accuracy: 0.7844 - val_loss: 0.3938 - learning_rate: 0.0010
Epoch 3/50
2293/2293 ━━━━━━━━━━━━━━━━━━━━ 152s 66ms/step - accuracy: 0.7804 - loss: 0.3992 - val_accuracy: 0.7958 - val_loss: 0.3635 - learning_rate: 0.0010
Epoch 4/50
1600/2293 ━━━━━━━━━━━━━━━━━━━━ 43s 63ms/step - accuracy: 0.8001 - loss: 0.3698

In [ ]:
model.save("./Models/Arabic/lstm.h5")  # Save model as .h5 file
print("✅ Model saved as my_model.h5")

In [ ]:
loss, accuracy = model.evaluate(X_test, y_test)


In [ ]:
print(f"Test Accuracy: {accuracy*100:.2f}%")


from sklearn.metrics import classification_report
y_pred = (model.predict(X_test) > 0.5).astype("int32")
print(classification_report(y_test, y_pred))

In [ ]:
import seaborn as sns

cm = confusion_matrix(y_test, y_pred)

plt.figure(figsize=(6, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=['Class 0', 'Class 1'], yticklabels=['Class 0', 'Class 1'])
plt.xlabel('Predicted')
plt.ylabel('True')
plt.title('Confusion Matrix')
plt.show()

In [ ]:
# Accuracy
plt.plot(history.history['accuracy'], label='Train Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Model Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend()
plt.grid()
plt.show()

# Loss
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Model Loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend()
plt.grid()
plt.show()

In [ ]:
from sklearn.metrics import roc_auc_score, roc_curve
import matplotlib.pyplot as plt


y_pred_proba = model.predict(X_test)


auc_score = roc_auc_score(y_test, y_pred_proba)
print(f"AUC Score: {auc_score:.4f}")

In [ ]:
fpr, tpr, thresholds = roc_curve(y_test, y_pred_proba)
plt.figure(figsize=(8, 6))
plt.plot(fpr, tpr, label=f"AUC = {auc_score:.4f}")
plt.plot([0, 1], [0, 1], 'k--') 
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.title("ROC Curve")
plt.legend(loc="lower right")
plt.grid()
plt.show()

In [ ]:
def predict_fake_news(text):
    sequence = tokenizer.texts_to_sequences([text])
    padded = pad_sequences(sequence, maxlen=300)  

    # Predict
    prediction = model.predict(padded)[0][0]
    label = "Fake" if prediction > 0.5 else "Real"

    print(f"Text: {text}")
    print(f"Prediction: {prediction:.4f} → {label}")

predict_fake_news("تم اكتشاف علاج جديد للسرطان، وهو معجزة علمية لم يسبق لها مثيل.")


> ## Train XLM Roberta Model

In [40]:


train_df, temp_df = train_test_split(df_ar, test_size=0.2, random_state=42)
val_df, test_df = train_test_split(temp_df, test_size=0.5, random_state=42)

train_dataset = Dataset.from_pandas(train_df)
val_dataset = Dataset.from_pandas(val_df)
test_dataset = Dataset.from_pandas(test_df)

# Load tokenizer and model
model_name = "xlm-roberta-base"
tokenizer = XLMRobertaTokenizer.from_pretrained(model_name)
model = XLMRobertaForSequenceClassification.from_pretrained(model_name, num_labels=len(df_ar['label'].unique()))

Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
def tokenize_function(examples):
    return tokenizer(examples['text'], padding="max_length", truncation=True, max_length=300)

train_dataset = train_dataset.map(tokenize_function, batched=True)
val_dataset = val_dataset.map(tokenize_function, batched=True)
test_dataset = test_dataset.map(tokenize_function, batched=True)

train_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'label'])
val_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'label'])
test_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'label'])

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = torch.argmax(torch.tensor(logits), dim=-1)
    accuracy = accuracy_score(labels, predictions)
    return {'accuracy': accuracy}

Map:   0%|          | 0/326032 [00:00<?, ? examples/s]

Map:   0%|          | 0/40754 [00:00<?, ? examples/s]

Map:   0%|          | 0/40754 [00:00<?, ? examples/s]

In [42]:
import torch
print(f"CUDA available: {torch.cuda.is_available()}")  
print(f"GPU count: {torch.cuda.device_count()}")       
print(f"Current device: {torch.cuda.current_device()}")  
print(f"Device name: {torch.cuda.get_device_name(0)}")

CUDA available: True
GPU count: 1
Current device: 0
Device name: Tesla P100-PCIE-16GB


In [ ]:
import wandb
wandb.login(key="your key here")

wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: mohamededderyouch5 (mohamededderyouch5-01) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

In [44]:
wandb.init(project="FND_project_arabic")

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


In [ ]:

from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir="results",
    eval_strategy="steps",        
    eval_steps=5000,                     
    save_strategy="steps",              
    save_steps=5000,
    save_total_limit=2,                 

    learning_rate=2e-5,
    per_device_train_batch_size=32,     
    per_device_eval_batch_size=32,
    gradient_accumulation_steps=2,       
    num_train_epochs=7,
    weight_decay=0.01,

    load_best_model_at_end=True,         
    metric_for_best_model="accuracy",    
    greater_is_better=True,              

    logging_dir="./logs",                
    logging_strategy="steps",           
    logging_steps=500,                   
    disable_tqdm=False,                  
    
    report_to="wandb",             
    fp16=True,                           
)

# Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics,
)

In [66]:

trainer.train()

Step,Training Loss,Validation Loss


KeyboardInterrupt: 

In [ ]:
test_results = trainer.evaluate(test_dataset)
print(test_results)

predictions, labels, _ = trainer.predict(test_dataset)
preds = torch.argmax(torch.tensor(predictions), dim=1)
print(classification_report(labels, preds, target_names=[str(i) for i in range(len(df_ar['label'].unique()))]))

In [ ]:
model.save_pretrained("./Models/Arabic/prerainmodel")

tokenizer.save_pretrained("./Models/Arabic/prerainmodel")

print("✅ Model and tokenizer saved.")